In [4]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import json
import sys
sys.path.append('../')
print(sys.path)
import prompts as p




# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

dataname = "multiterm_fix_fewshot_4-turbo-preview"

DATA_PATH = "../../../Datasets/MAMS-ATSA/Downsampled/test/test_multi_row_few_shot_fix.csv"
OUTPUT_PATH = f"../../../Datasets/Evaluations/Sentiment Analysis/{dataname}.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0



['/Users/timgutberlet/My Drive (t.gutberlet01@gmail.com)/05 - Coding/06 - Bachelor Thesis/Error-Analysis-with-LLMs/Sentiment Analysis/Pipelines/multi_term', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/timgutberlet/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '../', '../']


In [5]:
df = pd.read_csv(DATA_PATH)
prompt_name = "prompt_multi_term_fewshot"
prompt = p.prompt_multi_term_fewshot


df['polarity_pred'] = None
df['prompt_name'] = None
df['prompt'] = None
print(df.head())


   Unnamed: 0  text_id  term_id  \
0           0        0        0   
1           1        0        1   
2           2        0        2   
3           3        0        3   
4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive                 0   
1     dulce de leche ice-cream   neutral                 0   
2  chocolate sauce tic-tac-toe   neutral                 0   
3            poached pineapple   neutral                 0   
4                       server  negative                 0   

                                     example1_prompt  example2_term_id  \
0  In

In [6]:
def create_terms(df):
    s = ""
    for term in df['term']:
        s += "'"+ term + "'" + "\n"
    return s

def string_to_json(s):
    start_index = s.find('{')
    end_index = s.rfind('}') + 1
    json_string = s[start_index:end_index]
    json_data = json.loads(json_string)
    return json_data

In [7]:
def execute(df, prompt, prompt_name, start = 0):
    current_text = -1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    for i in range(start, len(df)):
        if df["text_id"][i] == current_text:
            continue
        current_text = df["text_id"][i]
        if(df["polarity_pred"][i] != None and df["polarity_pred"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        input_text = df["text"][i]
        terms_df = df[df["text_id"] == current_text]
        terms = create_terms(terms_df)
        example_prompt_1 = df['example1_prompt'][i]
        example_prompt_2 = df['example2_prompt'][i]
        example_prompt_3 = df['example3_prompt'][i]
        try: 
            result = chain.run(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms, callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'polarity_pred'] =  'error'
            df.loc[i, 'prompt_name'] = prompt_name
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        data = string_to_json(result)
        term_sentiments= data["term_sentiments"]
        x = 0
        for j in terms_df["term_id"]:
            if df["term"][j].lower() == term_sentiments[x][0].lower():
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            else:
                print(df["term"][j].lower())
                print(term_sentiments[x][0].lower())
                print(prompt_text)
                print("\n")
                print(result)
                print("\n")
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            x+=1
    return df

df = execute(df, prompt, prompt_name)
    



0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Example 1:
Input: "The decor is not special at all but their food and amazing prices make up for it."
Terms: 
'decor'
'food'
'prices'

Output: {"term_sentiments": [["decor", "negative"], ["food", "positive"], ["prices", "positive"]]}


Example 2:
Input: "when tables opened up, the manager sat another party before us."
Terms: 
'tables'
'manager'

Output: {"term_sentiments": [["tables", "neutral"], ["manager", "negative"]]}


Example 3
Input: "Though the menu includes some unorthodox offerings (a peanut butter roll, for instance), the classics are pure and great--we've never had better sushi anywhere, including Japan."
Terms: 
'menu'
'peanut butter roll'
'classics'
'sushi'

Output: {"term_sentiments": [["menu", "neutral"], ["peanut butter roll", "negative"], ["classics", "positive"], ["sushi", "positive"]]}

    
Task:
Input: Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea

KeyboardInterrupt: 

In [6]:
error_rows = df[df['polarity_pred'] == 'error']
print(error_rows)


Empty DataFrame
Columns: [Unnamed: 0, text_id, term_id, text, term, polarity, example1_term_id, example1_prompt, example2_term_id, example2_prompt, example3_term_id, example3_prompt, polarity_pred, prompt_name, prompt]
Index: []


In [7]:
df.head()

,Unnamed: 0,text_id,term_id,text,term,polarity,example1_term_id,example1_prompt,example2_term_id,example2_prompt,example3_term_id,example3_prompt,polarity_pred,prompt_name,prompt
0,0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,0,"Input: ""The decor is not special at all but th...",1,"Input: ""when tables opened up, the manager sat...",2,"Input: ""Though the menu includes some unorthod...",positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
1,1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,0,"Input: ""The decor is not special at all but th...",1,"Input: ""when tables opened up, the manager sat...",2,"Input: ""Though the menu includes some unorthod...",positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
2,2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,0,"Input: ""The decor is not special at all but th...",1,"Input: ""when tables opened up, the manager sat...",2,"Input: ""Though the menu includes some unorthod...",positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
3,3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,0,"Input: ""The decor is not special at all but th...",1,"Input: ""when tables opened up, the manager sat...",2,"Input: ""Though the menu includes some unorthod...",positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
4,4,1,4,The server was so busy the night we visited th...,server,negative,0,"Input: ""The decor is not special at all but th...",1,"Input: ""when tables opened up, the manager sat...",2,"Input: ""Though the menu includes some unorthod...",negative,prompt_multi_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."


In [43]:
df.to_csv(OUTPUT_PATH, index = True)